# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.tail(5)

,Treinamento,Relevância:,Treinamento Limpo
295,discord https://t.co/q6oxummoxf\nsiga nossas m...,0,discord https//t.co/q6oxummoxfsiga nossas midi...
296,se tivesse a ps5 despedia me da marta e dos me...,0,se tivesse a ps5 despedia me da marta e dos me...
297,"desculpa pai, mas ps5 vai ter que vir no final...",0,desculpa pai mas ps5 vai ter que vir no final ...
298,"eu nem tô assim, quem tá são vocês\n\n#ps5show...",0,eu nem tô assim quem tá são vocês#ps5showcase ...
299,as vezes dá medo de pensar que o #ps5 chegue n...,1,as vezes dá medo de pensar que o #ps5 chegue n...


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Teste,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1.0
1,hoje tem \n\n#ps5,0.0
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0.0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1.0
4,"@1playergirl bom, nesse momento me sinto num h...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [33]:
#Limpando caracteres:
def limpa_tweets(dataframe):
    
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w)
        w = w.replace('  ',' ')
        w = w.strip()
        w = w.casefold()
        ls.append(w)
    train['Treinamento Limpo'] = ls
    return train_limpo



In [26]:
#train.to_excel(r'PS5Emoji.xlsx', sheet_name='Treinamento', index = False)

## Classificador:
Precisamos de:
P(R|T) = P(T|R) * P(R)/ P(T)




In [7]:
#Calculando Probabilidades:
Rele = pd.Series(dtype=object)
Irrele = pd.Series(dtype=object)

for s in train['Treinamento Limpo']:
    lsWords = pd.Series(s.split())
    for i in range(len(lsWords)):
        if train['Relevância:'][i+1] == 1:
            Rele = Rele.append(lsWords)
        else:
            Irrele = Irrele.append(lsWords)


In [8]:
Rele2 = pd.Series(dtype=object)
Irrele2 = pd.Series(dtype=object)

for s in test['Teste']:
    lsWords = pd.Series(s.split())
    for i in range(len(lsWords)):
        if test['Relevância:'][i+1] == 1:
            Rele2 = Rele2.append(lsWords)
        else:
            Irrele2 = Irrele2.append(lsWords)

In [9]:
Rele2.describe()

count     60551
unique     1745
top           o
freq       2020
dtype: object

In [37]:
Irrele2.describe()

count     79644
unique     1745
top           o
freq       2723
dtype: object

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**